In [53]:
import boto3
import json

# login for boto3 credentials
boto3.setup_default_session(profile_name='root-AWSAdministratorAccess')
client = boto3.client('bedrock')
bedrock_runtime_client = boto3.client('bedrock-runtime')

In [54]:
# models details
response = client.list_foundation_models()
models = response['modelSummaries']
models_details = [f"{model['providerName']}, {model['modelName']}, {model['modelId']}, {model['modelArn']}, inputModalities: {','.join(model['inputModalities'])}, outputModalities: {','.join(model['outputModalities'])}" for model in models]
models_details.sort()
print('\n'.join(models_details))

AI21 Labs, J2 Grande Instruct, ai21.j2-grande-instruct, arn:aws:bedrock:us-east-1::foundation-model/ai21.j2-grande-instruct, inputModalities: TEXT, outputModalities: TEXT
AI21 Labs, J2 Jumbo Instruct, ai21.j2-jumbo-instruct, arn:aws:bedrock:us-east-1::foundation-model/ai21.j2-jumbo-instruct, inputModalities: TEXT, outputModalities: TEXT
AI21 Labs, Jurassic-2 Mid, ai21.j2-mid, arn:aws:bedrock:us-east-1::foundation-model/ai21.j2-mid, inputModalities: TEXT, outputModalities: TEXT
AI21 Labs, Jurassic-2 Mid, ai21.j2-mid-v1, arn:aws:bedrock:us-east-1::foundation-model/ai21.j2-mid-v1, inputModalities: TEXT, outputModalities: TEXT
AI21 Labs, Jurassic-2 Ultra, ai21.j2-ultra, arn:aws:bedrock:us-east-1::foundation-model/ai21.j2-ultra, inputModalities: TEXT, outputModalities: TEXT
AI21 Labs, Jurassic-2 Ultra, ai21.j2-ultra-v1, arn:aws:bedrock:us-east-1::foundation-model/ai21.j2-ultra-v1, inputModalities: TEXT, outputModalities: TEXT
Amazon, Titan Embeddings G1 - Text, amazon.titan-embed-text-v1, a

In [55]:
prompt  = 'list three names of people'
body = json.dumps(
                {
                    "prompt": prompt,
                    "maxTokens": 500,
                    "temperature": 0,
                    "stopSequences": [],
                })
response = bedrock_runtime_client.invoke_model(modelId='ai21.j2-ultra', body=body)
response_body = json.loads(response.get("body").read())
response_body

{'id': 1234,
 'prompt': {'text': 'list three names of people',
  'tokens': [{'generatedToken': {'token': '▁list',
     'logprob': -12.104637145996094,
     'raw_logprob': -12.104637145996094},
    'topTokens': None,
    'textRange': {'start': 0, 'end': 4}},
   {'generatedToken': {'token': '▁three',
     'logprob': -5.354193210601807,
     'raw_logprob': -5.354193210601807},
    'topTokens': None,
    'textRange': {'start': 4, 'end': 10}},
   {'generatedToken': {'token': '▁names▁of',
     'logprob': -12.85616397857666,
     'raw_logprob': -12.85616397857666},
    'topTokens': None,
    'textRange': {'start': 10, 'end': 19}},
   {'generatedToken': {'token': '▁people',
     'logprob': -5.3834547996521,
     'raw_logprob': -5.3834547996521},
    'topTokens': None,
    'textRange': {'start': 19, 'end': 26}}]},
 'completions': [{'data': {'text': '\nSure! Here are three names of people:\n\n1. John Doe\n2. Jane Smith\n3. David Johnson',
    'tokens': [{'generatedToken': {'token': '<|newline|>'